rpc permissions?


In [ ]:

// Always return the permission set so it can't be manipulated by another command.
function getPermissions() {
    return {
        'google contacts.ipynb[google contact settings]': 'Inner Circle'
    };
}
module.exports = getPermissions;


filter command permission?


In [ ]:
var importer = require('../Core');
var {
    getPermissions,
    getSettings,
    getDaysEvents,
    getOauthClient,
} = importer.import([
    'rpc permissions',
    'days events',
    'google contact settings',
    'import google calendar api'
]);

var options = {
    calendarId: 'Commands'
};

// TODO: move this logic out to a higher level coordinator?
function filterCommand(command, date, id, user) {
    const props = {};
    return getOauthClient(options)
        .then(() => getSettings(user))
        .then(settings => {
            // filter permissions
            Object.assign(props, settings);
            try {
                console.log(settings.circles)
                // compare id with permissions
                Object.assign(props, {
                    result: importer.interpret(command)
                });
                Object.assign(props, {
                    allowed: settings.circles
                        .filter(c => {
                            const matching = Object.keys(getPermissions())
                                .filter(k => getPermissions()[k].indexOf(c) > -1);
                            return matching.filter(k => importer.interpret(k).id === props.result.id).length > 0;
                        }).length > 0
                });
                // if looking for settings use the user name provided to this function for lookup
                // TODO: accept parameters from message context
                if(props.result.id === importer.interpret('google contacts.ipynb[google contact settings]').id) {
                    props.result.runInNewContext = () => settings;
                }
            } catch (e) {
                if((e + '').indexOf('Nothing found') > -1) {
                    Object.assign(props, {result: null});
                } else {
                    throw e;
                }
            }
        })
        .then(() => getDaysEvents(new Date(date), options))
        .then(events => {
            // filter already sent
            const already = events
              .filter(e => e.event.summary.indexOf('Result:') > -1 && e.event.summary.indexOf(id) > -1).length > 0;
            Object.assign(props, {already: already})
        })
        .catch(e => console.log(e))
        .then(() => props)
}
module.exports = filterCommand;


store rpc result?


In [ ]:
var importer = require('../Core');
var {
    updateEvent,
    ISODateString,
    getOauthClient,
} = importer.import([
    'convert date iso',
    'update create merge event',
    'import google calendar api'
]);

var options = {
    calendarId: 'Commands'
};

function storeResult(friend) {
    var result;
    if(typeof friend === 'undefined' || friend === null
       || friend.already !== false) {
        // skip commands that have already been run
        throw new Error('Nothing to do!');
    }
    
    var isEmail = false;
    if(typeof friend.subject !== 'undefined') {
        isEmail = true;
    }
    
    if (friend.result === null) {
        result = Promise.resolve('Command not found, please specify a command ' + (isEmail ? 'in the subject line' : '') + ' like "megamind settings" without quotes.')
    } else if(!friend.allowed) {
        result = Promise.resolve('Would have run "' + friend.result.id + '" but you don\'t have permission.');
    } else {
        const commandResult = friend.result.runInNewContext();
        result = Promise.resolve([])
            .then(() => typeof commandResult === 'function' ? commandResult() : commandResult)
            .catch(e => Object.getOwnPropertyNames(e).reduce((alt, key) => {
                alt[key] = e[key];
                return alt;
            }, {}))
    }

    var response;
    return getOauthClient(options)
        .then(() => result)
        .then(r => {
            response = r;
            // store as an event for tracking
            const config = {
                start: {
                    dateTime: ISODateString(new Date(friend.date.getTime()))
                },
                end: {
                    dateTime: ISODateString(new Date(friend.date.getTime() + 60 * 30 * 1000))
                },
                summary: 'Result: ' + (isEmail ? 'Email: ' : '') + friend.id,
                description: JSON.stringify(response, null, 4)
            }
            return updateEvent(config, options);
        })
        .then(() => response)
        .catch(e => console.log(e))
}
module.exports = storeResult;

